In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.nasnet import NASNetLarge,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexNoLegend"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (331,331,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(331, 331))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 331, 331, 3)

In [5]:
# Creating base_model
image_shape = (331, 331, 3)                                                                                                                                                                                                                                                                                            
base_model = NASNetLarge(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 4032)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 4032)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   51.9s


[CV] ........................... C=0.01, score=0.854207, total=  50.7s
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.824803, total=  50.4s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.848365, total=  50.2s
[CV] ........................... C=0.01, score=0.847676, total=  50.5s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.839286, total=  50.5s
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.841948, total=  50.7s
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.849602, total=  51.8s
[CV] C

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  3.0min


[CV] ............................ C=0.1, score=0.884314, total= 3.0min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.899602, total= 2.9min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.887017, total= 3.1min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.892430, total= 3.1min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.895153, total= 3.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.880906, total= 3.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.888889, total= 3.5min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.909002, total= 7.7min
[CV] ................... C=1000000000.0, score=0.886811, total= 7.6min
[CV] ................... C=1000000000.0, score=0.920588, total= 7.6min
[CV] ................... C=1000000000.0, score=0.912402, total= 6.3min
[CV] ................... C=1000000000.0, score=0.907738, total= 6.2min
[CV] ................... C=1000000000.0, score=0.893954, total= 6.3min
[CV] ................... C=1000000000.0, score=0.910979, total= 6.6min
[CV] ................... C=1000000000.0, score=0.902584, total= 6.5min
[CV] ................... C=1000000000.0, score=0.915339, total= 6.4min
[CV] .................. C=10000000000.0, score=0.900391, total= 5.0min
[CV] .................. C=10000000000.0, score=0.906067, total= 5.0min
[CV] .................. C=10000000000.0, score=0.907843, total= 5.0min
[CV] .................. C=10000000000.0, score=0.911417, total= 4.8min
[CV] .................. C=10000000000.0, score=0.907830, total= 4.8min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 34.6min remaining:  1.4min


[CV] .................. C=10000000000.0, score=0.884843, total= 5.2min
[CV] .................. C=10000000000.0, score=0.915925, total= 5.1min
[CV] .................. C=10000000000.0, score=0.906746, total= 4.8min
[CV] .................. C=10000000000.0, score=0.904573, total= 4.2min
[CV] .................. C=10000000000.0, score=0.909363, total= 4.2min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 35.3min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 50.26362803, 186.89578195, 399.80073712, 416.2185245 ,
        422.26230533, 450.38019774, 411.63090708, 449.51737044,
        412.32204206, 411.73488359, 438.19614816, 413.57709615,
        287.43072288]),
 'mean_score_time': array([0.07306066, 0.07531528, 0.07105813, 0.06562405, 0.06426971,
        0.06558819, 0.06582465, 0.06164062, 0.05794842, 0.05813401,
        0.05498157, 0.04406672, 0.02764022]),
 'mean_test_score': array([0.84066693, 0.88900947, 0.91061563, 0.90676796, 0.9043015 ,
        0.90301894, 0.90597869, 0.90489345, 0.90035517, 0.90213102,
        0.90568272, 0.90607735, 0.9054854 ]),
 'mean_train_score': array([0.8573731 , 0.9298319 , 0.984916  , 0.99594376, 0.99606476,
        0.99484802, 0.9970071 , 0.9965356 , 0.99295055, 0.99428708,
        0.99685419, 0.99681034, 0.99512267]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.84067, std: 0.00884, params: {'C': 0.01},
 mean: 0.88901, std: 0.00736, params: {'C': 0.1},
 mean: 0.91062, std: 0.00712, params: {'C': 1.0},
 mean: 0.90677, std: 0.00877, params: {'C': 10.0},
 mean: 0.90430, std: 0.00996, params: {'C': 100.0},
 mean: 0.90302, std: 0.00865, params: {'C': 1000.0},
 mean: 0.90598, std: 0.00814, params: {'C': 10000.0},
 mean: 0.90489, std: 0.00956, params: {'C': 100000.0},
 mean: 0.90036, std: 0.01364, params: {'C': 1000000.0},
 mean: 0.90213, std: 0.01197, params: {'C': 10000000.0},
 mean: 0.90568, std: 0.00927, params: {'C': 100000000.0},
 mean: 0.90608, std: 0.00961, params: {'C': 1000000000.0},
 mean: 0.90549, std: 0.00791, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 1.0} - score: 0.9106
